In this piece of code I simultaneously optimize over the unitary basis rotation and the mps tensors.

In [1]:
using Revise,LinearAlgebra, MPSKit, TensorKit, MPSKitExperimental
using GaussianBasis

LinearAlgebra.BLAS.set_num_threads(1);
TensorKit.TO.disable_cache();
TensorKit.Strided.disable_threads();

In [2]:
bset = BasisSet("6-31g", """
N        0.00      0.00     0.00      
N        2.118      0.00     0.00""")

6-31g Basis Set
Type: Spherical   Backend: Libcint

Number of shells: 10
Number of basis:  18

N: 1s 2s 1p 3s 2p 
N: 1s 2s 1p 3s 2p

I will keep the first 3 orbitals frozen filled, treat the next 10 orbitals using MPS, and keep the rest frozen free.

In [3]:
N_ORBS = bset.nbas;
N_frozen_filled = 4;
N_active = 12;
N_frozen_empty = N_ORBS-N_frozen_filled-N_active;
NELEC = 14;
MS2 = 0;

The state is now smaller:

In [42]:
psp = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((0,0,0)=>1, (1,1//2,1)=>1, (2,0,0)=>1);

left = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((-(NELEC-N_frozen_filled*2),MS2//2,mod1(-(NELEC-N_frozen_filled*2),2))=>1);
right = oneunit(left);

virtual = Vect[(Irrep[U₁]⊠Irrep[SU₂] ⊠ FermionParity)]((i,s,b)=>1 for i in -(NELEC-N_frozen_filled*2):0, s in 0:1//2:(MS2//2+1), b in (0,1));

st = FiniteMPS(rand,ComplexF64,N_active,psp,virtual;left,right);

The code works through a weird new type. We cannot work directly with an MPO hamiltonian, because I want to consider general unitary rotations between frozen orbitals and the active space. The MPO will only describe the active space, and therefore does not contain enough information.

In [63]:
nh = CASSCF_Ham(bset,N_frozen_filled+1:N_frozen_filled+N_active)

CASSCF_Ham{BasisSet{GaussianBasis.LCint, Molecules.Atom, SphericalShell}, Matrix{ComplexF64}, UnitRange{Int64}}(BasisSet{GaussianBasis.LCint, Molecules.Atom, SphericalShell}("6-31g", Molecules.Atom[Molecules.Atom{Float64, Float64}(7, 14.007, [0.0, 0.0, 0.0]), Molecules.Atom{Float64, Float64}(7, 14.007, [2.118, 0.0, 0.0])], SphericalShell[SphericalShell{Molecules.Atom{Float64, Float64}, 6, Float64}(0, [2.4069849440988125, 4.432663884944204, 7.161977226981375, 9.37348857044504, 8.029233024174598, 2.762149184707957], [4173.51146, 627.457911, 142.902093, 40.2343293, 12.8202129, 4.39043701], Molecules.Atom{Float64, Float64}(7, 14.007, [0.0, 0.0, 0.0])), SphericalShell{Molecules.Atom{Float64, Float64}, 3, Float64}(0, [-1.8287280250000597, -0.9040332377807428, 2.3847817471011106], [11.62636186, 2.716279807, 0.7722183966], Molecules.Atom{Float64, Float64}(7, 14.007, [0.0, 0.0, 0.0])), SphericalShell{Molecules.Atom{Float64, Float64}, 3, Float64}(1, [4.232585393498297, 3.295135320303207, 1.56464

Let's try it out!

In [64]:
# first some dmrg to ramp up the bond dimension
mpo_opp = MPSKitExperimental.mpo_representation(nh)[1]; # will be made easier in future
(st,env) = find_groundstate(st,mpo_opp,DMRG2(maxiter=3,trscheme=truncdim(20)),disk_environments(st,mpo_opp));

# grassmannscf simultaneously converges the state and the oribtals
@time (st,nh) = find_groundstate(st,nh,GrassmannSCF(maxiter=20));

┌ Info: Iteraton 0 error 0.2279050095708951
└ @ MPSKit /Users/maarten.vandamme/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:112


┌ Info: Iteraton 1 error 0.002759436394073167
└ @ MPSKit /Users/maarten.vandamme/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:112


┌ Info: Iteraton 2 error 0.00021746617003826074
└ @ MPSKit /Users/maarten.vandamme/.julia/packages/MPSKit/BToBw/src/algorithms/groundstate/dmrg.jl:112


┌ Info: CG: initializing with f = -90.393855064840, ‖∇f‖ = 1.9288e+01
└ @ OptimKit /Users/maarten.vandamme/.julia/packages/OptimKit/xpmbV/src/cg.jl:39


┌ Info: CG: iter    1: f = -97.609072639323, ‖∇f‖ = 1.3894e+00, α = 3.60e-02, β = 0.00e+00, nfg = 2
└ @ OptimKit /Users/maarten.vandamme/.julia/packages/OptimKit/xpmbV/src/cg.jl:83


...

┌ Info: CG: iter   17: f = -106.996568497756, ‖∇f‖ = 3.5191e-01, α = 1.20e-01, β = 6.37e-01, nfg = 2
└ @ OptimKit /Users/maarten.vandamme/.julia/packages/OptimKit/xpmbV/src/cg.jl:83


┌ Info: CG: iter   18: f = -107.017512589965, ‖∇f‖ = 9.0763e-01, α = 3.41e-01, β = 1.11e-01, nfg = 3
└ @ OptimKit /Users/maarten.vandamme/.julia/packages/OptimKit/xpmbV/src/cg.jl:83


┌ Info: CG: iter   19: f = -107.057910437832, ‖∇f‖ = 4.0071e-01, α = 1.00e-01, β = 6.67e+00, nfg = 2
└ @ OptimKit /Users/maarten.vandamme/.julia/packages/OptimKit/xpmbV/src/cg.jl:83


 79.797145 seconds (1.37 G allocations: 515.531 GiB, 23.23% gc time, 0.23% compilation time: 100% of which was recompilation)


┌ Warning: CG: not converged to requested tol: f = -107.077091453847, ‖∇f‖ = 7.9946e-01
└ @ OptimKit /Users/maarten.vandamme/.julia/packages/OptimKit/xpmbV/src/cg.jl:103


In practice, one should probably start from a well informed initial guess, instead of using atomic orbitals. Nevertheless, as a proof of principle, it works.